In [1]:
import numpy as np
import pandas as pd 
from scipy.stats import norm
import warnings as wn 
wn.filterwarnings('ignore')

This code is about CVR under the equity risk class.
CVR is applicable to: 
i) Options and embedded options
ii) Cashflows, which are not a linear function of notional


In [2]:
# Sample Portfolio data used for Curvature Risk Charge calculations
df = pd.read_excel(r'C:\Users\SAMIR SHARMA\Downloads\curvature_risk.xlsx')
df

,Option,bucket,rho,RW,S,K,r,T,sigma,type,position
0,1,5,0.25,0.3,100,90,0.05,1,0.20,call,long
1,2,5,0.25,0.3,50,50,0.05,2,0.15,call,short
2,3,5,0.25,0.3,150,155,0.05,3,0.18,call,long
3,4,8,0.25,0.5,88,85,0.05,1,0.16,put,long
4,5,8,0.25,0.5,210,200,0.05,2,0.22,put,short


Calculation methodology: Basically, this method asks us to shock the risk factors (equity prices in the case of equity classes) upward or downward,  measure the changes in PnL based on our pricing model, and then take out the effect of delta risk (loss due to a small change in the risk factor). It is trying to capture second-order risk (gamma) arising from option positions exposed to large movements in the underlying.

Important points to remember:
i) If the price of an instrument depends on several risk factors, the curvature risk must be determined separately for each risk factor.

ii) Risk factors in the case of the equity risk class are the spot prices.

iii) Sensitivities for each risk class are expressed in the reporting currency of the bank.

iv) Risk factor shocks are given as risk weights for each bucket.

v) In FRTB context, loss from CVR, i.e., positive CVR, comes from short gamma position (short call and short put have short gamma position), so we keep it. Gain from CVR, i.e., negative CVRcomes from a long gamma position (long call and long put have a long gamma position), so we drop it.
Tips: Think of CVR as a loss figure (which will help understand the above statement).


In [3]:
# this function uses BSM to price an option and gives value for each schocked scenarios(upward and downward)

def BSMPrice(RW, S, K, r, T, sigma, type='call', position='long', shock_scenario='base'):
    if shock_scenario == 'upward':
        S_effective = S * (1 + RW) # risk factor shocked upward according to bucket
    elif shock_scenario == 'downward':
        S_effective = S * (1 - RW) # risk factor shocked downward according to bucket
    else:
        S_effective = S
    
    # BSM inputs defined
    d1 = (np.log(S_effective / K) + (r + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if type == 'put' and position == 'long':
        put_price = norm.cdf(-d2) * K * np.exp(-r * T) - S_effective * norm.cdf(-d1)
        return put_price
    elif type == 'put' and position == 'short':
        put_price = norm.cdf(-d2) * K * np.exp(-r * T) - S_effective * norm.cdf(-d1)
        return -put_price
    elif type == 'call' and position == 'short':
        call_price = S_effective * norm.cdf(d1) - norm.cdf(d2) * K * np.exp(-r * T)
        return -call_price
    else:
        call_price = S_effective * norm.cdf(d1) - norm.cdf(d2) * K * np.exp(-r * T)
        return call_price
    

data = pd.read_excel(r'C:\Users\SAMIR SHARMA\Downloads\curvature_risk.xlsx')

df = pd.DataFrame(data)

d1_values = (np.log(df['S'] / df['K']) + (df['r'] + (df['sigma'] ** 2) / 2) * df['T']) / (df['sigma'] * np.sqrt(df['T']))
df['Nd1'] = norm.cdf(d1_values)
shock_scenarios = ['base', 'upward', 'downward']


for shock_scenario in shock_scenarios:
    df[shock_scenario + '_value'] = df.apply(lambda row: BSMPrice(row['RW'], row['S'], row['K'], row['r'], row['T'], row['sigma'],row['type'],row['position'],shock_scenario=shock_scenario), axis=1)
    
df

,Option,bucket,rho,RW,S,K,r,T,sigma,type,position,Nd1,base_value,upward_value,downward_value
0,1,5,0.25,0.3,100,90,0.05,1,0.20,call,long,0.809703,16.699448,44.529617,1.269830
1,2,5,0.25,0.3,50,50,0.05,2,0.15,call,short,0.718189,-6.838362,-19.963466,-0.462209
2,3,5,0.25,0.3,150,155,0.05,3,0.18,call,long,0.702580,27.063448,64.289435,4.657546
3,4,8,0.25,0.5,88,85,0.05,1,0.16,put,long,0.728832,2.545370,0.005026,36.854661
4,5,8,0.25,0.5,210,200,0.05,2,0.22,put,short,0.736892,-12.333995,-1.100807,-76.655446


In [4]:
# Function to calculate Curvature risk for two stress scenarios
# cvr_up and cvr_down means CVR+ and CVR- respectively as represented in BCBS MCR for market risk paper

def curvature_risk(RW,S,Nd1, base_value, upward_value, downward_value,type = 'call', position = 'long'):
    
    # short call give rise to short gamma position
    if type == 'call' and position == 'short':
        cvr_up =  -(upward_value - base_value - (-(Nd1)*RW*S))
        cvr_down =  -(downward_value - base_value + (-(Nd1)*RW*S))
        return cvr_up, cvr_down
    
    # long put gives rise to long gamma position
    elif type == 'put' and position == 'long':
        cvr_up =  -(upward_value - base_value - (Nd1-1)*RW*S)
        cvr_down =  -(downward_value - base_value + (Nd1-1)*RW*S)
        return cvr_up, cvr_down
    
    # short put gives rise to short gamma position
    elif type == 'put' and position == 'short':
        cvr_up =  -(upward_value - base_value - (-((Nd1-1))*RW*S))
        cvr_down =  -(downward_value - base_value + (-((Nd1-1))*RW*S))
        return cvr_up, cvr_down
    
    # long call gives rise to long gamma position
    else:
        cvr_up =  -(upward_value - base_value - (Nd1)*RW*S)
        cvr_down =  -(downward_value - base_value + (Nd1)*RW*S)
        return cvr_up, cvr_down
    
    
df['CVR'] = df.apply(lambda row: curvature_risk(row['RW'], row['S'], row['Nd1'], row['base_value'], row['upward_value'], row['downward_value'],row['type'],row['position']), axis=1)
df['cvr_up'] = df['CVR'].apply(lambda x: x[0])
df['cvr_down'] = df['CVR'].apply(lambda x: x[1])

df.drop('CVR', axis=1, inplace=True)
df

,Option,bucket,rho,RW,S,K,r,T,sigma,type,position,Nd1,base_value,upward_value,downward_value,cvr_up,cvr_down
0,1,5,0.25,0.3,100,90,0.05,1,0.20,call,long,0.809703,16.699448,44.529617,1.269830,-3.539077,-8.861473
1,2,5,0.25,0.3,50,50,0.05,2,0.15,call,short,0.718189,-6.838362,-19.963466,-0.462209,2.352266,4.396686
2,3,5,0.25,0.3,150,155,0.05,3,0.18,call,long,0.702580,27.063448,64.289435,4.657546,-5.609871,-9.210214
3,4,8,0.25,0.5,88,85,0.05,1,0.16,put,long,0.728832,2.545370,0.005026,36.854661,-9.391041,-22.377905
4,5,8,0.25,0.5,210,200,0.05,2,0.22,put,short,0.736892,-12.333995,-1.100807,-76.655446,16.393141,36.695121


### We have the required data above , now we move to aggregation part 
where we perform intrabucket and interbucket aggregation using appropriate correlation values